# Analisi - Grafi

In [1]:
import pickle
import json

import geopandas as gpd
import networkx as nx
from networkx.algorithms.approximation import steiner_tree
import osmnx as ox

from my_paths import *
import Graph_functions

## Analisi Automatica - Documentazione

Tramite la funzione Graph_functions.auto_analysis_poi() è possibile eseguire in automatico la ricreazione del grafo stradale con il dizionario dei pesi desiderato. Spiegazione della funzione:
- custom_weights: Un dizionario di pesi del tipo:
    ```python
    {"cycleway": 0, "primary": 1, ....}
    ```
    Possibilità di inserire come chiave: "default": quando non viene trovata una "highway" associta al dizionario,  
    verrà utilizzato il valore contenuto in "default".  
    In questo modo se si fornisce un dizionario in cui c'è soltanto la chiave "default", tutte le strade avranno quel  
    valore associato (esempio per creare un grafo di strade senza pesi):
    ```python
    {"default": 1}
    ```
    Se non viene dato nessun dizionario di pesi, verrà utilizzato direttamente il grafo salvato in staging, che contiene i pesi
    standard che abbiamo stabilito, salvati su file json.
- Si possono inserire i vari percorsi di salvataggio dei file pickle (grafo) e geojson(geoDataFrame).
    Richiede una struttura a liste di dizionari di geoDataFrame per "poi" del tipo:
    ```python
    gdf_list = [
        {"gdf":  gpd.read_file(PATH_DEL_GEOJSON),
        "tipo": "tipo che si vuole assegnare agli archi che connettono il gdf al grafo",
        "attr": {"nome_attributo": "valore_attributo"} # Opzionale, di solito non ci serve
        }
    ]
    ```

## Sport e tempo libero

Utilizzeremo i seguenti dati:
1. **Parchi**
2. **Impianti sportivi**
3. **Aree gioco**  

Saranno i nostri "poi", cioè "Point of Interest" che verranno aggiunti al Grafo della rete Ciclabile/Stradale per la ricerca di percorsi

In [2]:
gdf_list = [
    {"gdf":  gpd.read_file(PATH_PARCHI_CLEAN),
     "tipo": "parchi",
     "gdf":  gpd.read_file(PATH_IMPIANTI_SPORTIVI_CLEAN),
     "tipo": "parchi",
     "gdf":  gpd.read_file(PATH_AREE_GIOCO_CLEAN),
     "tipo": "parchi",}
]
Graph_functions.auto_analysis_poi(gdf_list, PATH_GEOJSON=PATH_SPORT_TEMPO_LIBERO_ANALISI_CLEAN)

### Esempio di statisiche

In [3]:
gdf_pesi = gpd.read_file(PATH_SPORT_TEMPO_LIBERO_ANALISI_CLEAN)

In [ ]:
# Eliminiamo connessioni "artificiali" usate per connettere i "poi" al grafo delle strade
gdf_pesi_mod = gdf_pesi[gdf_pesi["artificial"] != True]

# Eliminiamo i percorsi completamente contenuti all'interno dei "poi" poligonari: parchi
gdf = gpd.read_file(PATH_PARCHI_CLEAN)
geometry_parchi = gdf.union_all()
gdf_senza_parchi = []
for idx, row in gdf_pesi_mod.iterrows():
    if not geometry_parchi.contains(row.geometry):
        gdf_senza_parchi.append(row)
gdf_pesi_mod = gpd.GeoDataFrame(gdf_senza_parchi, crs=CRS_GRAD)

# Salviamo per test di visualizzazione su kepler
gdf_pesi_mod.to_file("../Data/Clean/Analisi/sport_senza_parchi_con_pesi.geojson")

# Raggruppiamo per "highway" e calcoliamo (in km -> /1000) la lunghezza complessiva delle strade
gdf_pesi_mod_most_highways = gdf_pesi_mod.to_crs(CRS_METR).groupby("highway").agg({
    "length": lambda x: sum(x/1000)
    }).sort_values(by="length", ascending=False)

In [ ]:
# Stampiamo il risultato
print(gdf_pesi_mod_most_highways)
print(f"Tot: {gdf_pesi_mod_most_highways["length"].sum()}")